Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

import camb
from camb import model, initialpower

In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='x-large'
plt.rc('font', **font)
plt.rcParams['legend.fontsize']='medium'

In [4]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [5]:
#Fiducial cosmological parameters
c=3e5
hubble=0.675
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9

nss = 0.9655
alphass = 0.0
betass = 0.0

gamma=0.545

kpivot = 0.05

In [6]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass, nrun=alphass, nrunrun=betass, pivot_scalar=kpivot)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [7]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [8]:
#Get matter power spectrum at z=0: P(k,z=0)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=1.0)
#Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-5, maxkh=0.5, npoints = 800)

In [9]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [10]:
#Derivatives

def dlogPd_dns(kk):
    return np.log(kk/kpivot)

def dlogPd_dalphas(kk):
    return (1./2)*pow(np.log(kk/kpivot),2)

def dlogPd_dbetas(kk):
    return (1./6)*pow(np.log(kk/kpivot),3)

In [11]:
#Redshift bins for SKA1-MID (Band 1)

zlist = np.arange(0.35,3.05,0.1)
ztest = 0.5
Nzbins = len(zlist)

print zlist
print "ztest =", ztest
print "Number of redshift bins =", Nzbins

[ 0.35  0.45  0.55  0.65  0.75  0.85  0.95  1.05  1.15  1.25  1.35  1.45
  1.55  1.65  1.75  1.85  1.95  2.05  2.15  2.25  2.35  2.45  2.55  2.65
  2.75  2.85  2.95]
ztest = 0.5
Number of redshift bins = 27


In [12]:
#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00
#Define the comoving distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

In [13]:
#Define the growth function in LCDM
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

print fg(ztest)

0.758362943343


In [14]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

print Dgz(ztest)

0.769389642041


In [15]:
#fiducial bHI from Bull et al 2015
def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

#fiducial OmHI Mario's fit
def OmHI(zc):
    return 0.00048+0.00039*zc-0.000065*pow(zc,2)

print bHI(ztest), OmHI(ztest)

0.7725 0.00065875


In [16]:
#Construct P(k,μ,z) 
def beta(zc):
    return fg(zc)/bHI(zc)

def Pkz(kk,mu,zc):    
    return pow(1+beta(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [17]:
#mean brightness temperature [mK] Mario's fit
def Tb(zc):
    return 0.0559+0.2324*zc-0.024*pow(zc,2)

print ztest, Tb(ztest)

0.5 0.1661


In [18]:
#Construct P_HI(k,μ,z), Tb assumed known
def PHI(kk,mu,zc):
    return pow(bHI(zc),2)*Pkz(kk,mu,zc)

print ztest

0.5


In [19]:
#noise specs

Ndishes=130 + 64
Ddish=15.0*100 #cm
Nbeams=1

def thetab(zc):
    return 21*(1+zc)/Ddish

def omegapix(zc):
    return 1.13*pow(thetab(zc),2)

fsky = 0.7
Area=fsky*41253 #deg^2
omegatot = Area*pow(pi/180,2)
ttotal = 5000*60*60 

def Tsys(zc):
    Tinst = 25.0*1e3 #mK
    fc = 1420.4/(1+zc)
    Tsky = 60*pow(300/fc,2.55)*1e3 #mK
    return Tinst + Tsky 

def tobs(zc):
    return ttotal*(omegapix(zc)/omegatot)*Ndishes*Nbeams

Dzbin = 0.1 #redshift bin width
dfpix = 50*1e3 #frequency resolution in Hz
midfreq = 1420.4e6 #Hz

def dzpix(zc):
    return pow(1+zc,2)*dfpix/midfreq
def sigpix(zc):
    return Tsys(zc)/np.sqrt(dfpix*tobs(zc)) 
def dVpixdz(zz):    
    return c*pow(rcom(zz),2)/(H00*Ez(zz))
def Vpix(zc):
    return omegapix(zc)*scipy.integrate.romberg(dVpixdz,zc-dzpix(zc)/2,zc+dzpix(zc)/2)

def Wsq(kk,mu,zc):
    #add small offset to avoid division by zero
    return 1e-20+np.exp(-pow(kk,2)*(1-mu**2)*pow(rcom(zc),2)*pow(thetab(zc),2)/(8*np.log(2)))

#note division by Tb^2
def Pnoise(kk,mu,zc):
    return pow(sigpix(zc),2)*Vpix(zc)*pow(Wsq(kk,mu,zc),-1.)/pow(Tb(zc),2)

In [20]:
#survey (bin) volume [Mpc^3]

def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

In [21]:
def kmin(zc):
    return 2*pi*pow(Vsur(zc),-1/3)

kNL = 0.1 #1/Mpc    
    
def kmax(zc):
    return kNL*pow(1+zc,2/(2+nss)) #non-linear cutoff 

print kmin(ztest),kmax(ztest)

0.00279771839403 0.131449794558


In [22]:
#effective volume going in the Fisher matrix
def Veff(kk,mu,zc):
    return Vsur(zc)*(PHI(kk,mu,zc)/(PHI(kk,mu,zc)+Pnoise(kk,mu,zc)))**2

In [23]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [24]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [25]:
mugrid = np.linspace(-1., 1., 513) 

In [26]:
#Fisher matrix parameters 
params = ["0:ns","1:alphas","2:betas"]

In [27]:
#%%time
#   Fisher matrix   # 

Npar = 2 #Npar = 3

#create array of zeros
s = Npar

Fishermat = np.zeros([s,s])

for zi in range(0,Nzbins):
    zc = zlist[zi]
    kgrid = np.linspace(kmin(zc), kmax(zc), 513)
    if kpivot < kmin(zc) or kpivot > kmax(zc): raise ValueError("kpivot out of range")
    K, MU = np.meshgrid(kgrid, mugrid)
    for i in range(0,Npar):  
        def deriv_i(kk,mu,zc):
            if i==0:  return dlogPd_dns(kk)
            elif i==1:  return dlogPd_dalphas(kk)
            elif i==2:  return dlogPd_dbetas(kk)
            else: print "out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlogPd_dns(kk)
                    elif j==1:  return dlogPd_dalphas(kk)
                    elif j==2:  return dlogPd_dbetas(kk)
                    else: print "index out of range" 
                Fishermat[i][j] += integrate2D(dF(K,MU),kgrid,mugrid)
            else: Fishermat[i,j] = Fishermat[j,i]

In [28]:
print Fishermat

[[ 219943.24188762   26231.04205861]
 [  26231.04205861   57704.94483958]]


In [29]:
#print covariance matrix
print linalg.inv(Fishermat)

[[  4.80724511e-06  -2.18523818e-06]
 [ -2.18523818e-06   1.83228851e-05]]


In [30]:
#print marginalised uncertainties
for i in range(0,Npar):
    print np.sqrt(linalg.inv(Fishermat)[i,i])

0.00219254306977
0.00428052392954
